# ReAct Agent with LangChain

This notebook demonstrates how to create and use a ReAct (Reasoning and Acting) agent with LangChain.

ReAct agents combine reasoning and acting by allowing the LLM to:
1. **Reason** about what action to take
2. **Act** by calling tools
3. **Observe** the results
4. Repeat until the task is complete

## What You'll Learn:
- Setting up a ReAct agent
- Integrating tools (Tavily Search)
- Running agent workflows
- Understanding agent decision-making process

## 1. Setup and Imports

In [1]:
# Import required libraries
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.agents.middleware import (
    SummarizationMiddleware,
    ToolRetryMiddleware,
    ModelFallbackMiddleware,
    LLMToolSelectorMiddleware,
)
from langgraph.checkpoint.memory import InMemorySaver
from langchain_openai import ChatOpenAI

# Optional backend helpers (installed when deploying)
try:
    from fastapi import FastAPI
    from langserve import add_routes
except ImportError:
    FastAPI = None
    add_routes = None

# Import our custom Tavily search tool
from tools.tavily_search import get_tavily_search_tool

# Load environment variables
load_dotenv()

print("✓ Imports successful!")

✓ Imports successful!


In [12]:
# Verify LangChain 1.0.5+ compatibility
import langchain

print(f"LangChain version: {langchain.__version__}")
if tuple(map(int, langchain.__version__.split('.')[:2])) >= (1, 0):
    print("✓ LangChain 1.0+ detected — all middleware and agent patterns are supported.")
else:
    print("⚠️ LangChain < 1.0 detected. Please upgrade: pip install -U langchain")

'1.0.5'

## 2. Set Up API Keys

You'll need:
- OpenAI API key (for the LLM)
- Tavily API key (for web search)

Set these as environment variables or in a `.env` file.

In [2]:
# Uncomment and set your API keys if not using .env file
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["TAVILY_API_KEY"] = "your-tavily-api-key"

# Verify keys are set
if "OPENAI_API_KEY" in os.environ and "TAVILY_API_KEY" in os.environ:
    print("✓ API keys are configured!")
else:
    print("⚠ Warning: Please set your API keys")

✓ API keys are configured!


## 3. Initialize the LLM

We'll use OpenAI's GPT-4 model as our reasoning engine.

In [3]:
# Initialize the LLM
# Note: Using model string (auto-inferred) or explicit ChatOpenAI instance
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Use gpt-4o-mini instead of gpt-5-nano for better cost/performance
    temperature=0.3,  # Lower temperature for more consistent reasoning
    max_tokens=2048,  # Explicit token limit
)

print(f"✓ LLM initialized: {llm.model_name}")

✓ LLM initialized: gpt-5-nano


## 4. Set Up Tools

Tools are the functions/APIs that the agent can use to interact with the world.
We'll use the Tavily search tool to enable web searching.

In [4]:
# Get the Tavily search tool
search_tool = get_tavily_search_tool(
    max_results=5,
    search_depth="advanced"
)

# Create tools list
tools = [search_tool]

print(f"✓ Tools configured: {[tool.name for tool in tools]}")

✓ Tools configured: ['tavily_search']


/home/nitish/Documents/github/Langchain/tools/tavily_search.py:92: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(


## 5. Create the ReAct Agent

The ReAct agent uses a specific prompt template that guides the LLM to:
- Think through problems step-by-step
- Decide which tools to use
- Process observations from tool outputs

### 5a. Middleware & Deployment Options
To give the ReAct agent general-purpose muscles, we can layer in LangChain's [built-in middleware](https://docs.langchain.com/oss/python/langchain/middleware/built-in) plus production backends from [LangGraph](https://docs.langchain.com/oss/python/langgraph/overview), [LangServe](https://python.langchain.com/docs/guides/langserve/), and [LangSmith](https://docs.langchain.com/langsmith/observability):
- **Summarization Middleware** keeps long chats within the context window by compressing older turns.
- **Tool Retry Middleware** automatically retries flaky tools (great for web search APIs).
- **Model Fallback Middleware** provides resilience by routing to backup models when the primary fails.
- **LLM Tool Selector Middleware** lets an auxiliary LLM pick the most relevant tools per turn, which is ideal once you expose many utilities.
- **LangGraph checkpointer** (InMemorySaver here) unlocks human-in-the-loop pauses and durable execution.
- **LangSmith tracing** gives observability/tracing with zero code once `LANGCHAIN_TRACING_V2=true`.
- **LangServe + FastAPI** exposes the agent as a REST endpoint so other services can call it.

In [ ]:
# Configure middleware and backend scaffolding (LangChain 1.0.5+)
checkpointer = InMemorySaver()
middleware_stack = [
    SummarizationMiddleware(
        model="gpt-5-nano",
        trigger={"tokens": 4000},  # Trigger after 4K tokens
        keep={"messages": 20},  # Keep last 20 messages
    ),
    ToolRetryMiddleware(
        max_retries=2,
        backoff_factor=1.5,
        initial_delay=1.0,
    ),
    ModelFallbackMiddleware(
        "gpt-5-nano",
        "gpt-4.1-mini",
    ),
    LLMToolSelectorMiddleware(
        model="gpt-4o-mini",
        max_tools=3,
        always_include=[tools[0].name] if tools else [],
    ),
]

backend_plan = {
    "langgraph": "InMemorySaver checkpoint enables pauses + resume for Human-in-the-loop flows.",
    "langsmith": "Set LANGCHAIN_TRACING_V2=true to stream traces, metrics, and alerts.",
    "langserve": "Optional FastAPI route to expose the agent via REST/WebSocket.",
}

print("✓ Middleware stack ready (LangChain 1.0.5+):")
for middleware in middleware_stack:
    print(f"  - {middleware.__class__.__name__}")

TypeError: SummarizationMiddleware.__init__() got an unexpected keyword argument 'trigger'

In [ ]:
### 5b. Checking Version Compatibility
Verify your LangChain version is 1.0.5 or later for full compatibility with this notebook.

In [9]:
# Create the ReAct agent with middleware (LangChain 1.0.5+ syntax)
# create_agent handles ReAct internally; no hub.pull() needed
agent = create_agent(
    model="gpt-4o-mini",
    tools=tools,
    middleware=middleware_stack,
    checkpointer=checkpointer,
    system_prompt="You are a helpful research assistant. Use available tools to find accurate information and provide comprehensive answers.",
)

print("✓ ReAct agent created (LangChain 1.0.5+ syntax)!")

✓ ReAct agent created (v1.0+ syntax)!


### 5b. Plug into LangSmith & LangServe
The [LangSmith observability stack](https://docs.langchain.com/langsmith/observability) streams every model/tool call when `LANGCHAIN_TRACING_V2=true`, and [LangServe](https://python.langchain.com/docs/guides/langserve/) wraps our agent executor in FastAPI routes so other services can invoke it over REST/WebSocket. Use this optional cell to verify tracing and expose an API when deploying.

In [ ]:
# Optional: wire up LangSmith tracing & LangServe deployment (LangChain 1.0.5+)
langsmith_enabled = os.environ.get("LANGCHAIN_TRACING_V2", "").lower() == "true"
if langsmith_enabled:
    print("✓ LangSmith tracing is ON — runs will stream to your project dashboard.")
else:
    print("ℹ Set LANGCHAIN_TRACING_V2=true (and LANGCHAIN_API_KEY/LANGCHAIN_PROJECT) to enable LangSmith traces.")

# Optional: LangServe FastAPI deployment
if FastAPI and add_routes:
    app = FastAPI(title="ReAct Agent Service", version="0.1.0")
    # In 1.0.5+, agents are Runnables — pass directly to add_routes
    add_routes(app, agent, path="/react-agent")
    print("✓ LangServe route mounted at /react-agent (launch with `uvicorn app:app --reload`).")
else:
    print("ℹ Install `fastapi` and `langserve` to expose this agent over REST/WebSocket.")

## 6. Example 1: Simple Information Retrieval

Let's ask the agent to find current information.

In [ ]:
# Run the agent with a query using v1.0 syntax
# Invoke with messages dict (not 'input' dict)
result = agent.invoke({
    "messages": [{"role": "user", "content": "What are the latest developments in artificial intelligence in 2024?"}]
})

print("\n" + "="*80)
print("FINAL ANSWER:")  
print("="*80)
# In v1.0+, the last message contains the agent's response
if result["messages"]:
    last_msg = result["messages"][-1]
    print(last_msg.content if hasattr(last_msg, 'content') else last_msg)

## 7. Example 2: Multi-Step Reasoning

Now let's try a more complex query that requires multiple reasoning steps.

In [ ]:
# Complex query requiring research and synthesis
result = agent.invoke({
    "messages": [{"role": "user", "content": "Who is the current CEO of OpenAI and what is their background?"}]
})

print("\n" + "="*80)
print("FINAL ANSWER:")
print("="*80)
if result["messages"]:
    last_msg = result["messages"][-1]
    print(last_msg.content if hasattr(last_msg, 'content') else last_msg)

## 8. Example 3: Comparative Analysis

The agent can perform comparative analysis by gathering multiple pieces of information.

In [ ]:
# Comparative query
result = agent.invoke({
    "messages": [{"role": "user", "content": "Compare the main features of GPT-4 and Claude 3. What are the key differences?"}]
})

print("\n" + "="*80)
print("FINAL ANSWER:")
print("="*80)
if result["messages"]:
    last_msg = result["messages"][-1]
    print(last_msg.content if hasattr(last_msg, 'content') else last_msg)

## 9. Understanding the ReAct Loop

The ReAct agent follows this pattern:

```
Thought: I need to find information about X
Action: tavily_search
Action Input: "X latest information"
Observation: [Search results]
Thought: Based on the results, I can answer...
Final Answer: [Agent's response]
```

This loop continues until the agent determines it has enough information to provide a final answer.

## 10. Custom Query

Try your own query!

In [ ]:
# Your custom query here
custom_query = "What is LangChain and how is it used in AI applications?"

result = agent.invoke({
    "messages": [{"role": "user", "content": custom_query}]
})

print("\n" + "="*80)
print("FINAL ANSWER:")
print("="*80)
if result["messages"]:
    last_msg = result["messages"][-1]
    print(last_msg.content if hasattr(last_msg, 'content') else last_msg)

## Summary

In this notebook, you learned:
- ✓ How to set up a ReAct agent with LangChain
- ✓ How to integrate tools (Tavily search) with agents
- ✓ How the ReAct reasoning loop works
- ✓ How to use agents for information retrieval and analysis

## Next Steps
- Explore adding more tools to the agent
- Try different LLM models
- Experiment with custom prompts
- Check out `01.deep_agents_basic.ipynb` for more advanced agent patterns